In [2]:
%load_ext autoreload
%autoreload 2

# `Logit` on Orders - A warm-up challenge (~1h)

## Select features

🎯 Let's figure out the impact of `wait_time` and `delay_vs_expected` on very `good/bad reviews`

👉 Using our `orders` training_set, we will run two `multivariate logistic regressions`:
- `logit_one` to predict `dim_is_one_star` 
- `logit_five` to predict `dim_is_five_star`.

 

In [3]:
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

👉 Import your dataset:

In [27]:
from olist.order import Order
orders = Order().get_training_data(with_distance_seller_customer=True)


In [26]:
orders.head()

,order_id,wait_time,expected_wait_time,delay_vs_expected,order_status,dim_is_five_star,dim_is_one_star,review_score,number_of_products,number_of_sellers,price,freight_value,distance_seller_customer
0,e481f51cbdc54678b7cc49136f2d6af7,8.436574,15.544063,0.0,delivered,0,0,4,1,1,29.99,8.72,18.063837
1,53cdb2fc8bc7dce0b6741e2150273451,13.782037,19.137766,0.0,delivered,0,0,4,1,1,118.70,22.76,856.292580
2,47770eb9100c2d0c44946d9cf07ec65d,9.394213,26.639711,0.0,delivered,1,0,5,1,1,159.90,19.22,514.130333
3,949d5b44dbf5de918fe9c16f97b45f8a,13.208750,26.188819,0.0,delivered,1,0,5,1,1,45.00,27.20,1822.800366
4,ad21c59c0840e6cb83a9ceb5573f8159,2.873877,12.112049,0.0,delivered,1,0,5,1,1,19.90,8.72,30.174037


👉 Select in a list which features you want to use:

⚠️ Make sure you are not creating data leakage (i.e. selecting features that are derived from the target)

💡 To figure out the impact of `wait_time` and `delay_vs_expected` we need to control for the impact of other features, include in your list all features that may be relevant

In [5]:
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif


🕵🏻 Check the `multi-colinearity` of your features, using the `VIF index`.

* It shouldn't be too high (< 10 preferably) to ensure that we can trust the partial regression coefficents and their associated `p-values` 
* Do not forget to standardize your data ! 
    * A `VIF Analysis` is made by regressing a feature vs. the other features...
    * So you want to `remove the effect of scale` so that your features have an equal importance before running any linear regression!
    
    
📚 <a href="https://www.statisticshowto.com/variance-inflation-factor/">Statistics How To - Variance Inflation Factor</a>

📚  <a href="https://online.stat.psu.edu/stat462/node/180/">PennState - Detecting Multicollinearity Using Variance Inflation Factors</a>

⚖️ Standardizing:

In [40]:
features = [
    "wait_time",
    "delay_vs_expected",
    "number_of_products",
    "number_of_sellers",
    "price", 
    "freight_value", 
    "distance_seller_customer"
]


In [41]:
orders_standardized = orders.copy()
for f in features:
    mu = orders[f].mean()
    sigma = orders[f].std()
    orders_standardized[f] = orders[f].map(lambda x: (x - mu) / sigma)
orders_standardized.head()

,order_id,wait_time,expected_wait_time,delay_vs_expected,order_status,dim_is_five_star,dim_is_one_star,review_score,number_of_products,number_of_sellers,price,freight_value,distance_seller_customer
0,e481f51cbdc54678b7cc49136f2d6af7,-0.431192,15.544063,-0.161781,delivered,0,0,4,-0.264595,-0.112544,-0.513802,-0.652038,-0.979475
1,53cdb2fc8bc7dce0b6741e2150273451,0.134174,19.137766,-0.161781,delivered,0,0,4,-0.264595,-0.112544,-0.086640,0.000467,0.429743
2,47770eb9100c2d0c44946d9cf07ec65d,-0.329907,26.639711,-0.161781,delivered,1,0,5,-0.264595,-0.112544,0.111748,-0.164053,-0.145495
3,949d5b44dbf5de918fe9c16f97b45f8a,0.073540,26.188819,-0.161781,delivered,1,0,5,-0.264595,-0.112544,-0.441525,0.206815,2.054621
4,ad21c59c0840e6cb83a9ceb5573f8159,-1.019535,12.112049,-0.161781,delivered,1,0,5,-0.264595,-0.112544,-0.562388,-0.652038,-0.959115


👉 Run your VIF Analysis to analyze the potential multicolinearities:

In [42]:
stan_feat=orders_standardized[features]

In [43]:
stan_feat.head()

,wait_time,delay_vs_expected,number_of_products,number_of_sellers,price,freight_value,distance_seller_customer
0,-0.431192,-0.161781,-0.264595,-0.112544,-0.513802,-0.652038,-0.979475
1,0.134174,-0.161781,-0.264595,-0.112544,-0.086640,0.000467,0.429743
2,-0.329907,-0.161781,-0.264595,-0.112544,0.111748,-0.164053,-0.145495
3,0.073540,-0.161781,-0.264595,-0.112544,-0.441525,0.206815,2.054621
4,-1.019535,-0.161781,-0.264595,-0.112544,-0.562388,-0.652038,-0.959115


In [44]:
df=pd.DataFrame()
df['featur']= stan_feat.columns
df['vif_index']= [vif(stan_feat.values, i) for i in range(stan_feat.shape[1])]
round(df.sort_values(by='vif_index' , ascending=False), 2)

,featur,vif_index
0,wait_time,2.62
1,delay_vs_expected,2.21
5,freight_value,1.67
6,distance_seller_customer,1.44
2,number_of_products,1.37
4,price,1.21
3,number_of_sellers,1.09


## Logistic Regressions

👉 Fit two `Logistic Regression` models:
- `logit_one` to predict `dim_is_one_star` 
- `logit_five` to predict `dim_is_five_star`.

`Logit 1️⃣`

In [45]:
' + '.join(features)

'wait_time + delay_vs_expected + number_of_products + number_of_sellers + price + freight_value + distance_seller_customer'

In [50]:
logit1= smf.logit(formula='dim_is_one_star ~ '+ ' + '.join(features),data= orders_standardized).fit()
logit1.summary()

Optimization terminated successfully.
         Current function value: 0.273582
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:        dim_is_one_star   No. Observations:                95872
Model:                          Logit   Df Residuals:                    95864
Method:                           MLE   Df Model:                            7
Date:                Fri, 01 Dec 2023   Pseudo R-squ.:                  0.1448
Time:                        20:29:56   Log-Likelihood:                -26229.
converged:                       True   LL-Null:                       -30669.
Covariance Type:            nonrobust   LLR p-value:                     0.000
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   -2.4676      0.013   -190.852      0.000      -2.493      -2.442
wait_time                    0.7124      0.017     42.173      0.000       0.679       0.746
delay_vs_expected            0.2553      0.019     13.801      0.000       0.219       0.292
number_of_products           0.2500      0.011     23.634      0.000       0.229       0.271
number_of_sellers            0.1778      0.008     22.642      0.000       0.162       0.193
price                        0.0505      0.011      4.464      0.000       0.028       0.073
freight_value               -0.0219      0.013     -1.697      0.090      -0.047       0.003
distance_seller_customer    -0.1785      0.014    -12.934      0.000      -0.206      -0.151
============================================================================================
"""

`Logit 5️⃣`

In [52]:
logit2= smf.logit(formula='dim_is_five_star ~ '+ ' + '.join(features),data= orders_standardized).fit()
logit2.summary()

Optimization terminated successfully.
         Current function value: 0.636830
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:       dim_is_five_star   No. Observations:                95872
Model:                          Logit   Df Residuals:                    95864
Method:                           MLE   Df Model:                            7
Date:                Fri, 01 Dec 2023   Pseudo R-squ.:                 0.05806
Time:                        20:30:50   Log-Likelihood:                -61054.
converged:                       True   LL-Null:                       -64817.
Covariance Type:            nonrobust   LLR p-value:                     0.000
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                    0.3388      0.007     47.340      0.000       0.325       0.353
wait_time                   -0.5219      0.012    -44.693      0.000      -0.545      -0.499
delay_vs_expected           -0.4330      0.024    -18.414      0.000      -0.479      -0.387
number_of_products          -0.1365      0.008    -16.331      0.000      -0.153      -0.120
number_of_sellers           -0.1427      0.008    -18.219      0.000      -0.158      -0.127
price                        0.0208      0.008      2.721      0.007       0.006       0.036
freight_value                0.0053      0.009      0.590      0.555      -0.012       0.023
distance_seller_customer     0.0872      0.008     10.509      0.000       0.071       0.103
============================================================================================
"""

💡 It's time to analyse the results of these two logistic regressions:

- Interpret the partial coefficients in your own words.
- Check their statistical significances with `p-values`
- Do you notice any differences between `logit_one` and `logit_five` in terms of coefficient importances?

In [53]:
# Among the following sentences, store the ones that are true in the list below

a = "delay_vs_expected influences five_star ratings even more than one_star ratings"
b = "wait_time influences five_star ratings even more more than one_star"

your_answer = [a]

🧪 __Test your code__

In [54]:
from nbresult import ChallengeResult

result = ChallengeResult('logit',
    answers = your_answer
)
result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /home/parissa/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /home/parissa/code/ParissaPeimaniyfard/04-Decision-Science/04-Logistic-Regression/data-logit/tests
plugins: typeguard-2.13.3, anyio-3.6.2, asyncio-0.19.0
asyncio: mode=strict
collecting ... collected 1 item

test_logit.py::TestLogit::test_question PASSED                           [100%]

============================== 1 passed in 0.02s ===============================


💯 You can commit your code:

git add tests/logit.pickle

git commit -m 'Completed logit step'

git push origin master



<details>
    <summary>- <i>Explanations and advanced concepts </i> -</summary>


> _All other thing being equal, the `delay factor` tends to increase the chances of getting stripped of the 5-star even more so than it affect the chances of 1-star reviews. Probably because 1-stars are really targeting bad products themselves, not bad deliveries_
    
❗️ However, to be totally rigorous, we have to be **more careful when comparing coefficients from two different models**, because **they might not be based on similar populations**!
    We have 2 sub-populations here: (people who gave 1-stars; and people who gave 5-stars) and they may exhibit intrinsically different behavior patterns. It may well be that "happy-people" (who tends to give 5-stars easily) are less sensitive as "grumpy-people" (who shoot 1-stars like Lucky-Luke), when it comes to "delay", or "price"...

</details>


## Logistic vs. Linear ?

👉 Compare the coefficients obtained from:
- A `Logistic Regression` to explain `dim_is_five_star`
- A `Linear Regression` to explain `review_score` 

Make sure to use the same set of features for both regressions.  

⚠️ Check that both sets of coefficients  tell  "the same story".

> YOUR ANSWER HERE

In [55]:
logit2= smf.logit(formula='dim_is_five_star ~ '+ ' + '.join(features),data= orders_standardized).fit()
logit2.summary()

Optimization terminated successfully.
         Current function value: 0.636830
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:       dim_is_five_star   No. Observations:                95872
Model:                          Logit   Df Residuals:                    95864
Method:                           MLE   Df Model:                            7
Date:                Fri, 01 Dec 2023   Pseudo R-squ.:                 0.05806
Time:                        20:43:43   Log-Likelihood:                -61054.
converged:                       True   LL-Null:                       -64817.
Covariance Type:            nonrobust   LLR p-value:                     0.000
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                    0.3388      0.007     47.340      0.000       0.325       0.353
wait_time                   -0.5219      0.012    -44.693      0.000      -0.545      -0.499
delay_vs_expected           -0.4330      0.024    -18.414      0.000      -0.479      -0.387
number_of_products          -0.1365      0.008    -16.331      0.000      -0.153      -0.120
number_of_sellers           -0.1427      0.008    -18.219      0.000      -0.158      -0.127
price                        0.0208      0.008      2.721      0.007       0.006       0.036
freight_value                0.0053      0.009      0.590      0.555      -0.012       0.023
distance_seller_customer     0.0872      0.008     10.509      0.000       0.071       0.103
============================================================================================
"""

In [57]:
lin_reg= smf.ols(formula='review_score ~ '+ ' + '.join(features),data= orders_standardized).fit()
lin_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           review_score   R-squared:                       0.145
Model:                            OLS   Adj. R-squared:                  0.145
Method:                 Least Squares   F-statistic:                     2322.
Date:                Fri, 01 Dec 2023   Prob (F-statistic):               0.00
Time:                        20:44:56   Log-Likelihood:            -1.5258e+05
No. Observations:               95872   AIC:                         3.052e+05
Df Residuals:                   95864   BIC:                         3.052e+05
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                    4.1555      0.004   1082.730      0.000       4.148       4.163
wait_time                   -0.4397      0.006    -70.719      0.000      -0.452      -0.428
delay_vs_expected           -0.0515      0.006     -9.023      0.000      -0.063      -0.040
number_of_products          -0.1297      0.004    -28.861      0.000      -0.139      -0.121
number_of_sellers           -0.1314      0.004    -32.747      0.000      -0.139      -0.124
price                       -0.0029      0.004     -0.696      0.486      -0.011       0.005
freight_value                0.0043      0.005      0.868      0.385      -0.005       0.014
distance_seller_customer     0.0967      0.005     20.984      0.000       0.088       0.106
==============================================================================
Omnibus:                    18749.799   Durbin-Watson:                   2.009
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            37820.902
Skew:                          -1.175   Prob(JB):                         0.00
Kurtosis:                       4.986   Cond. No.                         3.02
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

🏁 Congratulations! 

💾 Don't forget to commit and push your `logit.ipynb` notebook !